In [3]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd

sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/')
sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/utils')
sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/models')

from utils.analysis import *
from utils.getters import * 
from utils.plotters import * 
from models_config.model_config_GSP import *
from models_config.model_config_GSP_emb import *
import matplotlib.pyplot as plt

plt.rcParams['text.latex.preamble']=[r"\usepackage{lmodern}"]
params = {
    'text.usetex' : True,
    'font.size' : 11,
    'font.family' : 'lmodern'
}
plt.rcParams.update(params)


/var/folders/1j/5s36wz6s0jlb1k64vsq1k2zh0000gn/T/ipykernel_21602/4118881939.py:18: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  plt.rcParams['text.latex.preamble']=[r"\usepackage{lmodern}"]


In [2]:
def get_mean_CV_metric_student_model(dataset, model, analysis_type, training_type, view, run, student, dataset_split, model_args):

    import sklearn.metrics as metrics

    #get the mean metric for a student for a particular CV training_type 
    
    student_acc, student_recall, student_precision, student_f1 = 0, 0, 0, 0
    acc_mean, recall_mean, precision_mean, f1_mean = [], [], [], []
    
    if training_type == "3Fold":
        cv_number = 3
    if training_type == "5Fold":
        cv_number = 5
    if training_type == "10Fold":
        cv_number = 10  
    
    for i in range(cv_number):
        x = get_labels_and_preds(dataset=dataset, 
                                model=model,
                                analysis_type=analysis_type, 
                                training_type=training_type,  
                                cv_n=i, 
                                view=view, 
                                run=run, 
                                dataset_split=dataset_split, 
                                student=student, 
                                model_args=model_args)
        result = {
            'prec': metrics.precision_score(x['labels'],  x['preds']),
            'recall': metrics.recall_score(x['labels'],  x['preds']),
            'acc': metrics.accuracy_score(x['labels'],  x['preds']),
            'F1': metrics.f1_score(x['labels'],  x['preds'])
        }   
        acc_mean.append(result['acc'])
        recall_mean.append(result['recall'])
        precision_mean.append(result['prec'])
        f1_mean.append(result['F1'])
    
    student_acc = np.mean(acc_mean)
    student_recall = np.mean(recall_mean)
    student_precision = np.mean(precision_mean)
    student_f1 = np.mean(f1_mean)

    return student_acc, student_f1, student_recall, student_precision

In [3]:
def get_student_model_metric_all_folds(dataset, model, CV, runs, analysis_type, dataset_split, view, model_args):
    #Get average student modele metrics across all runs and all cv for all models in ensemble 
    """
    USAGE:
    runs = [i for i in range(10)]
    CV=["3Fold", "5Fold", "10Fold"]
    model = "gcn_student_ensamble_3"
    analysis_type="model_assessment"
    model_args= gcn_student_ensamble_args
    dataset_split="val"
    view=2

    get_student_model_metric_all_folds(dataset, model, CV, runs, analysis_type, dataset_split, view, model_args)
    """

    all_student_metrics_mean = []
    all_student_metrics_var = []

    for student in range(model_args["n_students"]):

        model_metrics_runs = []
        
        for run in runs:
                
            model_metrics = []
            
            for training_type in CV:
                
                model_metrics.append(get_mean_CV_metric_student_model(
                    dataset=dataset, 
                    model=model, 
                    analysis_type=analysis_type, 
                    training_type=training_type, 
                    view=view, 
                    run=run, 
                    student=student, 
                    dataset_split=dataset_split,
                    model_args=model_args
                )
                )
            
            model_metrics = np.mean(model_metrics, axis=0)
            model_metrics_runs.append(model_metrics)
        all_student_metrics_var.append(np.var(model_metrics_runs, axis=0))
        all_student_metrics_mean.append(np.mean(model_metrics_runs, axis=0))

    return all_student_metrics_mean, all_student_metrics_var


In [4]:
def get_best_student_ensamble(model, view, dataset, CV, runs, dataset_split, analysis_type, model_args):
    import warnings
    warnings.filterwarnings('ignore')
    
    student_rep = []
    student_var = []

    # get the best reproducibility across all runs 
    for run in runs:

        mean_all_runs = []

        for student in range(model_args["n_students"]):
            view_rep, _ = view_reproducibility_analysis(
                dataset=dataset, 
                models=[model], 
                CV=CV, 
                views=[view], 
                run=run, 
                students=[student], 
                model_args=[model_args]
            )
            mean_all_runs.append(view_rep[0][0])
        #store the reproducibility score for all students for the run 
        student_rep.append(mean_all_runs)
    
    #get the mean and variance for the reproducibility scores for all the students
    student_var = np.var(student_rep, axis=0)
    student_rep = np.mean(student_rep, axis=0)

    #get the metrics of all the student models
    all_student_metrics_mean, _ = get_student_model_metric_all_folds(
        dataset=dataset, 
        model=model, 
        CV=CV, 
        runs=runs, 
        analysis_type=analysis_type, 
        dataset_split=dataset_split, 
        view=view, 
        model_args=model_args
    )
    
    #get the best model based soley on max reproducibility 
    best_rep_max = 0
    metric = 0
    student_max = -1
    for i, rep in enumerate(student_rep):
        if rep > metric:
            best_rep_max = rep 
            metric = rep
            student_max = i

    #get the best model based on max reproducibility and accuracy
    best_rep_acc = 0
    metric = 0
    student_acc_index = -1
    for i, rep in enumerate(student_rep):
        student_acc = all_student_metrics_mean[i][0]
        if (rep+student_acc)/2 > metric:
            metric = (rep+student_acc)/2 
            best_rep_acc = rep
            student_acc_index = i
    
    #get the best model based on max reproducibility and f1 score
    best_rep_f1 = 0
    metric = 0
    student_f1_index = -1
    for i, rep in enumerate(student_rep):
        student_f1 = all_student_metrics_mean[i][1]
        if (rep+student_f1)/2 > metric:
            metric = (rep+student_f1)/2 
            best_rep_f1 = rep
            student_f1_index = i
    
    return [best_rep_max, student_var[student_max], student_max], [best_rep_acc, student_var[student_acc_index], student_acc_index], [best_rep_f1, student_var[student_f1_index], student_f1_index]
    

## Overview of the models and ensamble 

In [5]:
def get_student_model_metric(dataset, model, CV, runs, analysis_type, dataset_split, view, model_args):
    #Get average student modele metrics across all runs and all cv for all models in ensemble 
    #EACH FOLD INDIVIDUAL

    student_data_mean = [] 
    student_data_var = [] 
    
    for student in range(model_args["n_students"]):

        run_data = []   

        for run in runs:

            model_metrics = []
            
            for training_type in CV:

                model_cv_metrics = get_mean_CV_metric_student_model(
                    dataset=dataset, 
                    model=model, 
                    analysis_type=analysis_type, 
                    training_type=training_type, 
                    view=view, 
                    run=run, 
                    student=student, 
                    dataset_split=dataset_split,
                    model_args=model_args
                    )
                model_metrics.append(model_cv_metrics)
            
            run_data.append(model_metrics)
        
        student_data_mean.append(np.mean(run_data, axis=0))   
        student_data_var.append(np.std(run_data, axis=0))        

    return student_data_mean, student_data_var

In [6]:
def get_model_metric(dataset, model, CV, runs, analysis_type, dataset_split, view, model_args):
    #Get average metrics across all runs and all cv for any model 
    #EACH FOLD INDIVIDUAL

    data_mean = [] 
    data_var = [] 

    run_data = []   


    for run in runs:

        model_metrics = []
        
        for training_type in CV:
            
            model_cv_metrics = get_mean_CV_metric_student_model(
                dataset=dataset, 
                model=model, 
                analysis_type=analysis_type, 
                training_type=training_type, 
                view=view, 
                run=run, 
                student=-1, 
                dataset_split=dataset_split,
                model_args=model_args
                )
            model_metrics.append(model_cv_metrics)
        
        run_data.append(model_metrics)
        
    data_mean.append(np.mean(run_data, axis=0))   
    data_var.append(np.std(run_data, axis=0))        

    return data_mean, data_var

# REPRODUCIBILITY

In [7]:
def view_reproducibility_analysis_student_specific(dataset, models, CV, views, run, students=[0], model_args=None):
    """
    Reproducibility analysis for a single run for specific students in ensamble 
    student length = number of views, it is the specific student for each view
    """

    view_data_mean = []
    view_data_std = []

    for i, view in enumerate(views):
        
        model_result_mean = []
        model_result_std = []
        ensamble_count = 0

        for j, model in enumerate(models):

            rep_score, std = view_specific_rep(dataset=dataset, view=view, model=model, run=run, CV=CV, student=students[ensamble_count][i], model_args=model_args[j])
            model_result_mean.append(rep_score)
            model_result_std.append(std)
            
            if "ensamble" in models:
                ensamble_count += 1

        view_data_mean.append(model_result_mean)
        view_data_std.append(model_result_std)

    view_data_std.append(list(np.std(view_data_mean, axis=0)))
    view_data_std = np.array(view_data_std).T

    view_data_mean.append(list(np.mean(view_data_mean, axis=0)))

    view_data_mean = np.array(view_data_mean).T
    
    return view_data_mean, view_data_std 

In [28]:
def plot_bar_chart_reproducibility_mulitple_runs_student_specific(dataset, views, models, CV, runs, students=0, model_args=None, save_fig=False):
    """
    USAGE:
    plot_bar_chart_reproducibility_mulitple_runs_student_specific(
    dataset="gender_data", 
    views=[0, 2, 4, 5], 
    models=[  
        "gcn", 
        "gcn_student",
        "gcn_student_teacher",
        "gcn_student_ensamble_3", 
    ], 
    CV=["3Fold", "5Fold", "10Fold"], 
    runs=[i for i in range(10)], 
    students=[2,0,2,0], 
    model_args= [
        gcn_args,
        gcn_student_args,
        gcn_student_args,
        gcn_student_ensamble_args,
    ], 
    save_fig=False
    )
    """
    plt.rcParams["figure.figsize"] = (20,8)

    barWidth = 1/(len(models)+1)

    mean_all_runs = []
    views=[0, 2, 4, 5]
    runs=[i for i in range(10)]
    for run in runs:
        view_data_mean, _ = view_reproducibility_analysis_student_specific(dataset, models, CV, views, run, students=students, model_args=model_args)
        mean_all_runs.append(view_data_mean)

    mean_all_std = np.std(mean_all_runs, axis=0)
    mean_all_runs = np.mean(mean_all_runs, axis=0)
    
    X = np.arange(len(views)+1)

    sep = 0.00
    for i, view_d in enumerate(mean_all_runs):
        if models[i] == "gcn":
            plt.bar(X + sep, view_d, yerr=mean_all_std[i], capsize=4, width = barWidth, edgecolor ='grey', label=models[i]+"_teacher", alpha=0.5)
        elif models[i] == "gcn_student_ensamble_3":
            plt.bar(X + sep, view_d, yerr=mean_all_std[i], capsize=4, width = barWidth, edgecolor ='grey', label="best_student_ensemble", alpha=0.5)
        else:
            plt.bar(X + sep, view_d, yerr=mean_all_std[i], capsize=4, width = barWidth, edgecolor ='grey', label=models[i], alpha=0.5)
        
        sep += barWidth

    max_y_lim = 1 if np.amax(mean_all_runs) + np.max(mean_all_std) > 1 else np.amax(mean_all_runs) + np.max(mean_all_std)
    min_y_lim = 0 if np.amin(mean_all_runs) - np.max(mean_all_std) - 0.01 < 0 else np.amin(mean_all_runs) - np.max(mean_all_std) - 0.01
    plt.ylim(min_y_lim, max_y_lim)

    title = f"Reproducibility Score for Dataset:{dataset} across {len(runs)} different seeds for best student"

    plt.ylabel("Reproducibility Score")
    x_ticks = ["View {}".format(i) for i in views]+ ["Average"]

    plt.xticks([r + barWidth for r in range(len(mean_all_runs[0]))], x_ticks)
    plt.title(title)
    plt.grid(axis = 'y')
    plt.legend()
    if save_fig:
        if not os.path.exists(SAVE_DIR_FIGS+"ensemble_results/"):
            os.makedirs(SAVE_DIR_FIGS+"ensemble_results/")
        
        plt.savefig(SAVE_DIR_FIGS+"ensemble_results/"+title+".png", dpi=150,bbox_inches='tight')
        plt.clf()
    
    else:
        plt.show()
        plt.clf()  

In [29]:
def get_best_student_ensamble_detailed(model, view, CV, runs, dataset_split, analysis_type, model_args):
    import warnings
    warnings.filterwarnings('ignore')
    
    student_rep = []
    student_var = []

    # get the best reproducibility across all runs 
    for run in runs:

        mean_all_runs = []

        for student in range(model_args["n_students"]):
            view_rep, _ = view_reproducibility_analysis(
                dataset=dataset, 
                models=[model], 
                CV=CV, 
                views=[view], 
                run=run, 
                students=[student], 
                model_args=[model_args]
            )
            mean_all_runs.append(view_rep[0][0])
        #store the reproducibility score for all students for the run 
        student_rep.append(mean_all_runs)
    
    #get the mean and variance for the reproducibility scores for all the students
    student_var = np.var(student_rep, axis=0)
    student_rep = np.mean(student_rep, axis=0)

    #get the metrics of all the student models
    all_student_metrics_mean, all_student_metrics_var = get_student_model_metric_all_folds(
        dataset=dataset, 
        model=model, 
        CV=CV, 
        runs=runs, 
        analysis_type=analysis_type, 
        dataset_split=dataset_split, 
        view=view, 
        model_args=model_args
    )
    
    #get the best model based soley on max reproducibility 
    best_rep_max = 0
    metric = 0
    student_max = -1
    for i, rep in enumerate(student_rep):
        if rep > metric:
            best_rep_max = rep 
            metric = rep
            student_max = i

    #get the best model based on max accuracy
    best_max_acc = 0
    metric = 0
    student_max_acc_index = -1
    for i, rep in enumerate(student_rep):
        student_acc = all_student_metrics_mean[i][0]
        if student_acc > metric:
            metric = student_acc
            best_max_acc = rep
            student_max_acc_index = i

    #get the best model based on max reproducibility and f1 score
    best_max_f1 = 0
    metric = 0
    student_max_f1_index = -1
    for i, rep in enumerate(student_rep):
        student_f1 = all_student_metrics_mean[i][1]
        if student_f1 > metric:
            metric = student_f1
            best_max_f1 = rep
            student_max_f1_index = i

    #get the best model based on max reproducibility and accuracy
    best_rep_acc = 0
    metric = 0
    student_acc_index = -1
    for i, rep in enumerate(student_rep):
        student_acc = all_student_metrics_mean[i][0]
        if (rep+student_acc)/2 > metric:
            metric = (rep+student_acc)/2 
            best_rep_acc = rep
            student_acc_index = i
    
    #get the best model based on max reproducibility and f1 score
    best_rep_f1 = 0
    metric = 0
    student_f1_index = -1
    for i, rep in enumerate(student_rep):
        student_f1 = all_student_metrics_mean[i][1]
        if (rep+student_f1)/2 > metric:
            metric = (rep+student_f1)/2 
            best_rep_f1 = rep
            student_f1_index = i
    
    return all_student_metrics_mean, all_student_metrics_var, [[best_rep_max, student_var[student_max], student_max], [best_max_acc, student_var[student_max_acc_index], student_max_acc_index], [best_max_f1, student_var[student_max_f1_index], student_max_f1_index], [best_rep_acc, student_var[student_acc_index], student_acc_index], [best_rep_f1, student_var[student_f1_index], student_f1_index]]
    

In [5]:
def extract_weights(dataset, view, model, run, student, model_args=None):
    weights = []
    for cv_n in range(3):
        weights.append(get_weight(dataset, view, model, "3Fold", 0, cv_n, run, student, model_args=model_args))
    for cv_n in range(5):
        weights.append(get_weight(dataset, view, model, "5Fold", 0, cv_n, run, student, model_args=model_args))
    for cv_n in range(10):
        weights.append(get_weight(dataset, view, model, "10Fold", 0, cv_n, run, student, model_args=model_args))
    
    return weights

In [9]:
dataset_split="val"
dataset = "gender_data"

dis = []

for run in range(10):

    all_views_dis = []
    all_views_no_dis = []

    for view in [0,2,4,5]:

        dis_view = []
        
        for student in [0,1,2,3]:
            dis_view.append(extract_weights(dataset=dataset, 
                            view=view, 
                            model=gcn_student_lsp_ensamble_4_args["model_name"], 
                            run=run, 
                            student=student, 
                            model_args=gcn_student_lsp_ensamble_4_args))
        
        all_views_dis.append(dis_view)

    dis.append(all_views_dis)


In [ ]:
#2,1,2,1

In [74]:
x = [np.abs(np.mean(dis[0][0][2],axis=0)), np.abs(np.mean(dis[0][1][1],axis=0)), np.abs(np.mean(dis[0][2][2],axis=0)), np.abs(np.mean(dis[0][3][1],axis=0))]

In [78]:
x = np.mean(x, axis=0)

In [79]:
x

array([0.13072726, 0.13951984, 0.11718448, 0.05603635, 0.15121764,
       0.12207612, 0.08546476, 0.11959788, 0.10563517, 0.07958572,
       0.09139051, 0.07531274, 0.15088941, 0.12047654, 0.06034484,
       0.07263748, 0.05835674, 0.04134506, 0.08323871, 0.04535076,
       0.05926897, 0.1346688 , 0.08365707, 0.16205174, 0.17513928,
       0.06115662, 0.09999743, 0.12043419, 0.09372913, 0.08389772,
       0.05551011, 0.07964937, 0.050348  , 0.08112493, 0.02001393],
      dtype=float32)

In [80]:
np.argsort(-x)

array([24, 23,  4, 12,  1, 21,  0,  5, 13, 27,  7,  2,  8, 26, 28, 10,  6,
       29, 22, 18, 33, 31,  9, 11, 15, 25, 14, 20, 16,  3, 30, 32, 19, 17,
       34])

In [11]:
labels = [
        "Bank of the Superior Temporal Sulcus",#1
        "Caudal Anterior-cingulate Cortex",#2
        "Caudal Middle Frontal Gyrus",#3
        "Unmeasured Corpus Callosum",#4
        "Cunesus Cortex",#5
        "Entorhinal Cortex",#6
        "Fusiform Gyrus",#7
        "Inferior Parietal Cortex",#8
        "Inferior Temporal Gyrus",#9
        "Isthmus-cingulate Cortex",#10
        "Lateral occipital cortex",#11
        "Lateral orbital frontal cortex",#12
        "Lingual gyrus",#13
        "Medial orbital frontal cortex",#14
        "Middle temporal gyrus",#15
        "Parahippocampal gyrus",#16
        "Paracentral lobule",#17
        "Pars opercularis",#18
        "Pars orbitalis",#19
        "Pars triangularis",#20
        "Pericalcarine cortex",#21
        "Postcentral gyrus",#22
        "Posterior-cingulate cortex",#23
        "Precentral gyrus",#24
        "Precuneus cortex",#25
        "Rostral anterior cingulate cortex",#26
        "Rostral middle frontal gyrus",#£7
        "Superior frontal gyrus",#28
        "Superior parietal cortex",#29
        "Superior temporal gyrus",#30
        "Supramarginal gyrus",#31
        "Frontal pole",#32
        "Temporal pole",#33
        "Transverse temporal cortex",#34
        "Insula cortex"#35
]

In [81]:
labels[24],  labels[23],  labels[4],  labels[12],  labels[1]

('Precuneus cortex',
 'Precentral gyrus',
 'Cunesus Cortex',
 'Lingual gyrus',
 'Caudal Anterior-cingulate Cortex')

In [ ]:
#cingulate cortex, medial and lateral frontal cortex, temporoparietal regions, insula, and precuneus.

In [ ]:
#TODO DO FOR ALL OF THE MODELS AND SEE IF THEY FIND THE SAME REGIONS 

In [83]:
from models_config.model_config_GSP import *

dataset_split="val"
dataset = "gender_data"

dis = []

for run in range(10):

    all_views_dis = []
    all_views_no_dis = []

    for view in [0,2,4,5]:

        dis_view = []
        
        for student in [0,1,2,3]:
            dis_view.append(extract_weights(dataset=dataset, 
                            view=view, 
                            model=gcn_gat_student_lsp_ensamble_4_args["model_name"], 
                            run=run, 
                            student=student, 
                            model_args=gcn_gat_student_lsp_ensamble_4_args))
        
        all_views_dis.append(dis_view)

    dis.append(all_views_dis)


In [84]:
x = [np.abs(np.mean(dis[0][0][1],axis=0)), np.abs(np.mean(dis[0][1][3],axis=0)), np.abs(np.mean(dis[0][2][1],axis=0)), np.abs(np.mean(dis[0][3][3],axis=0))]
x = np.mean(x, axis=0)
np.argsort(-x)

array([ 5, 10, 14, 24, 33, 25, 18, 29, 21,  9, 27, 32,  7,  1,  4, 12, 23,
       16, 17, 30,  6, 11, 20,  3, 26, 13, 34, 22, 19,  8, 31, 28,  2,  0,
       15])

In [93]:
a = [5, 10, 14, 24, 33, 25, 18, 29, 21,  9, 27, 32,  7,  1,  4, 12, 23, 16]
b = [24, 23,  4, 12,  1, 21,  0,  5, 13, 27,  7,  2,  8, 26, 28, 10,  6, 29]
list(set(a) & set(b))

[1, 4, 5, 7, 10, 12, 21, 23, 24, 27, 29]

In [90]:
from models_config.model_config_GSP import *

dataset_split="val"
dataset = "gender_data"

dis = []

for run in range(10):

    all_views_dis = []
    all_views_no_dis = []

    for view in [0,2,4,5]:

        dis_view = []
        
        for student in [0,1]:
            dis_view.append(extract_weights(dataset=dataset, 
                            view=view, 
                            model=gat_gat_student_lsp_ensamble_2_args["model_name"], 
                            run=run, 
                            student=student, 
                            model_args=gat_gat_student_lsp_ensamble_2_args))
        
        all_views_dis.append(dis_view)

    dis.append(all_views_dis)


In [91]:
x = [np.abs(np.mean(dis[0][0][1],axis=0)), np.abs(np.mean(dis[0][1][1],axis=0)), np.abs(np.mean(dis[0][2][1],axis=0)), np.abs(np.mean(dis[0][3][0],axis=0))]
x = np.mean(x, axis=0)
np.argsort(-x)

array([ 4, 10, 14, 21, 23, 27,  7, 22, 30,  9, 25,  5, 12,  6, 11, 24, 33,
       20, 29, 28,  3, 26, 34,  1, 18,  8, 32,  2, 16, 15, 31, 13, 17, 19,
        0])

In [94]:
c = [4, 10, 14, 21, 23, 27,  7, 22, 30,  9, 25,  5, 12,  6, 11, 24, 33, 20]
d = [1, 4, 5, 7, 10, 12, 21, 23, 24, 27, 29]
list(set(c) & set(d))

[4, 5, 7, 10, 12, 21, 23, 24, 27]

In [8]:
from models_config.model_config_GSP import *

dataset_split="val"
dataset = "gender_data"

dis = []

for run in range(10):

    all_views_dis = []
    all_views_no_dis = []

    for view in [0,2,4,5]:

        dis_view = []
        
        for student in [0,1,2,3]:
            dis_view.append(extract_weights(dataset=dataset, 
                            view=view, 
                            model=gat_student_lsp_ensamble_4_args["model_name"], 
                            run=run, 
                            student=student, 
                            model_args=gat_student_lsp_ensamble_4_args))
        
        all_views_dis.append(dis_view)

    dis.append(all_views_dis)


In [9]:
[3,1,2,1]
x = [np.abs(np.mean(dis[0][0][3],axis=0)), np.abs(np.mean(dis[0][1][1],axis=0)), np.abs(np.mean(dis[0][2][1],axis=0)), np.abs(np.mean(dis[0][3][1],axis=0))]
x = np.mean(x, axis=0)
np.argsort(-x)

array([23,  7, 12, 29, 27, 18, 26, 16, 13,  1, 22,  9, 28, 11, 20,  6, 33,
       14,  3, 24, 15, 34, 25, 32,  8, 19,  2, 10,  5, 21, 31, 30,  0, 17,
        4])

In [10]:
c = [23,  7, 12, 29, 27, 18, 26, 16, 13,  1, 22,  9, 28, 11, 20,  6, 33, 14]
d = [4, 5, 7, 10, 12, 21, 23, 24, 27]
list(set(c) & set(d))

[27, 12, 23, 7]

In [12]:
labels[27],  labels[12],  labels[23],  labels[7]

('Superior frontal gyrus',
 'Lingual gyrus',
 'Precentral gyrus',
 'Inferior Parietal Cortex')